In [24]:
## Imports

# utility modules
import glob
import os
import sys
import re

# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt

# specialty modules
import h5py
import pyproj

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
sys.path.append("/home/jovyan/surface_velocity")

In [26]:
import pointCollection as pc
from readers.read_HDF5_ATL03 import read_HDF5_ATL03
from readers.get_ATL03_x_atc import get_ATL03_x_atc

## 1.3 Geographic setting

Let's read the the Mosaic of Antarctica for the region around Pine Island Glacier.  The mosaic is in polar-stereographic coordinates, so we'll need to project the geographic coordinates of the box into that projection to decide what part of the mosaic to read.

In [27]:
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
MOA=pc.grid.data().from_geotif(os.path.join('/srv/tutorial-data/land_ice_applications/' 'MOA','moa_2009_1km.tif'), bounds=[XR, YR])

# show the mosaic:
plt.figure()
MOA.show(cmap='gray', clim=[14000, 17000])
plt.plot(xy[0,:], xy[1,:])
plt.title('Mosaic of Antarctica for Pine Island Glacier')

[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356950.,  183825.,  561825.]), 'origin': 'lower'}


Text(0.5, 1.0, 'Mosaic of Antarctica for Pine Island Glacier')

In [28]:
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

# 3. Repeat track data from ATL06
Next, let's look at the big picture, combining data from multiple tracks around Pine Island Glacier.  We'll use the projected coordinates and plot on top of the image mosaic.

First, we'll every 25th point from one of the center-beam pairs for all files in our data directory.  We'll print an error if the reading fails, but will let the code continue anyway:

In [30]:
# find all the files in the directory:
#ATL06_files=glob.glob(os.path.join(data_root, 'PIG_ATL06', '*.h5'))
data_root='/srv/shared/surface_velocity/'
ATL06_files=glob.glob(os.path.join(data_root, 'FIS_ATL06_small', '*.h5'))
D_dict={}
error_count=0
for file in ATL06_files:
    try:
        D_dict[file]=atl06_to_dict(file, '/gt2l', index=slice(0, -1, 25), epsg=3031)
    except KeyError as e:
        print(f'file {file} encountered error {e}')
        error_count += 1
print(f"read {len(D_dict)} data files of which {error_count} gave errors")

read 13 data files of which 0 gave errors


About 20 of the files had problems, likely because clouds obscured the surface for gt2l.  That's too bad, but we can still work with the data we have. 

## 3.1. Repeat structure by cycle

Let's map the ground tracks for cycles 1 and 2 (not on repeat tracks) and for cycles 3 and later (measured on the repeat tracks).

In [31]:
plt.figure(figsize=[8,8])
hax0=plt.gcf().add_subplot(211, aspect='equal')
MOA.show(ax=hax0, cmap='gray', clim=[14000, 17000]);
hax1=plt.gcf().add_subplot(212, aspect='equal', sharex=hax0, sharey=hax0)
MOA.show(ax=hax1, cmap='gray', clim=[14000, 17000]);
for fname, Di in D_dict.items():
    cycle=Di['cycle']
    if cycle <= 2:
        ax=hax0
    else:
        ax=hax1
    #print(fname)
    #print(f'\t{rgt}, {cycle}, {region}')
    ax.plot(Di['x'], Di['y'])
    if True:
        try:
            if cycle  < 3:
                ax.text(Di['x'][0], Di['y'][0], f"rgt={Di['rgt']}, cyc={cycle}", clip_on=True)
            elif cycle==3:
                ax.text(Di['x'][0], Di['y'][0], f"rgt={Di['rgt']}, cyc={cycle}+", clip_on=True)
        except IndexError:
            pass
hax0.set_title('cycles 1 and 2');
hax1.set_title('cycle 3+');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356950.,  183825.,  561825.]), 'origin': 'lower'}
{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356950.,  183825.,  561825.]), 'origin': 'lower'}


Notice that cycles 1 and 2 were not precicely repeated, but all data from cycles 3 and onwards follow an exact set of repeats.  We've labeled the start of each track.  In this area, ascending tracks are labeled on the right (true South) side of the plot, while descending tracks are labeled on the left (true North) side.

## 3.2 A map-view look at elevations
Now let's map the elevations associated with these plots.

In [32]:
map_fig=plt.figure()
map_ax=map_fig.add_subplot(111)
MOA.show(ax=map_ax, cmap='gray', clim=[14000, 17000])
for fname, Di in D_dict.items():
    # select elevations with good quality_summary
    good=Di['atl06_quality_summary']==0
    ms=map_ax.scatter( Di['x'][good], Di['y'][good],  2, c=Di['h_li'][good], \
                  vmin=0, vmax=1000, label=fname)
map_ax._aspect='equal'
plt.colorbar(ms, label='elevation');


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356950.,  183825.,  561825.]), 'origin': 'lower'}


Elevations run from low (blue) on the ice shelf, to high (yellow) on the surrounding ridges. There are a few elevation blunders here, but nothing to cry about, and most of the tracks seem to have fairly good coverage.

## 3.3 Plotting a repeat-track elevation profile
Next, let's plot a single profile for the black line in the figure above.  We'll plot the surface height (h_li) as a function of the along-track coordinate, x_atc.  Since when we read in the elevations to make the survey plot we only read every 100th elevation, we'll reread the data at full resolution before plotting it:

In [33]:
D_2l={}
D_2r={}

# specify the rgt here:
rgt="1092"
# iterate over the repeat cycles
for cycle in ['03','04','05','06','07']:
    for filename in glob.glob(os.path.join(data_root, 'FIS_ATL06_small', f'*ATL06_*_{rgt}{cycle}*_003*.h5')):
        try:
            # read the left-beam data
            D_2l[filename]=atl06_to_dict(filename,'/gt2l', index=None, epsg=3031)
            # read the right-beam data
            D_2r[filename]=atl06_to_dict(filename,'/gt2r', index=None, epsg=3031)
            # plot the locations in the previous plot
            map_ax.plot(D_2r[filename]['x'], D_2r[filename]['y'],'k');  
            map_ax.plot(D_2l[filename]['x'], D_2l[filename]['y'],'k');
        except Exception as e:
            print(f'filename={filename}, exception={e}')

plt.figure();
for filename, Di in D_2l.items():
    #Plot only points that have ATL06_quality_summary==0 (good points)
    #hl=plot_segs(Di, ind=Di['atl06_quality_summary']==0, label=f"cycle={Di['cycle']}")
    hl=plt.plot(Di['x_atc'][Di['atl06_quality_summary']==0], Di['h_li'][Di['atl06_quality_summary']==0], '.', label=f"cycle={Di['cycle']}")

plt.legend()
plt.xlabel('x_atc')
plt.ylabel('elevation');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We see here that we have elevations from cycles 3, 5, and 6, but cycle 4 is missing.  That's likely because of clouds, and it's a fact of life in places where glaciers are found.

In [40]:
a=[]
ACC=[]
for filename, Di in D_2l.items():
    a.append(filename)
length=len(D_2l[a[0]]['h_li'][Di['atl06_quality_summary']==0])-1000
for offset in range(-500,500):
    ACC.append(np.corrcoef(D_2l[a[0]]['h_li'][Di['atl06_quality_summary']==0][500+offset:500+offset+length],D_2l[a[1]]['h_li'][Di['atl06_quality_summary']==0][500:500+length])[1,0])
dist=(np.argmax(ACC)-500)*(D_2l[a[0]]['x_atc'][Di['atl06_quality_summary']==0][1]-D_2l[a[0]]['x_atc'][Di['atl06_quality_summary']==0][0])
print("Distance travelled between passes: %f metres" % dist)
print("Correlation coefficent is %f" % ACC[np.argmax(ACC)])
#At this point, take dist*distance_interval/GPS_time_between_passes and that's our velocity

Distance travelled between passes: 79.596139 metres
Correlation coefficent is 0.999970
